# MCP ToolSpec

This tool connects to MCP Servers and allows an Agent to call the tools provided by MCP Servers.

This idea is migrated from [Integrate MCP tools into LlamaIndex](https://psiace.me/posts/integrate-mcp-tools-into-llamaindex/).

In [ ]:
from llama_index.llms.azure_openai import AzureOpenAI
import os

llm = AzureOpenAI(
    model=os.environ.get("AZURE_OPENAI_MODEL", "your-model-name"),
    max_tokens=int(os.environ.get("AZURE_OPENAI_MAX_TOKENS", 4096)),
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT", "your-azure-endpoint"),
    api_key="your-api-key",
    api_version="your-api-version",
    engine="your-deployment-name",
)

In [ ]:
from mcp.client.session import ClientSession
from mcp.client.sse import sse_client
from mcp.client.stdio import stdio_client, StdioServerParameters

from urllib.parse import urlparse
from contextlib import asynccontextmanager


class MCPClient(ClientSession):
    def __init__(
        self, command_or_url: str, args: list[str] = [], env: dict[str, str] = {}
    ):
        self.command_or_url = command_or_url
        self.args = args
        self.env = env

    @asynccontextmanager
    async def _run_session(self):
        if urlparse(self.command_or_url).scheme in ("http", "https"):
            async with sse_client(self.command_or_url) as streams:
                async with ClientSession(*streams) as session:
                    await session.initialize()
                    yield session
        else:
            server_parameters = StdioServerParameters(
                command=self.command_or_url, args=self.args, env=self.env
            )
            async with stdio_client(server_parameters) as streams:
                async with ClientSession(*streams) as session:
                    await session.initialize()
                    yield session

    async def call_tool(self, tool_name: str, arguments: dict):
        async with self._run_session() as session:
            return await session.call_tool(tool_name, arguments)

    async def list_tools(self):
        async with self._run_session() as session:
            return await session.list_tools()

In [ ]:
from llama_index.tools.mcp import McpToolSpec
from llama_index.core.llms import ChatMessage
from llama_index.core.agent import ReActAgent, ReActChatFormatter
from llama_index.core.agent.react.prompts import REACT_CHAT_SYSTEM_HEADER

SYSTEM_PROMPT = """\
You are an AI assistant.

Before you answer the user's question, you need to fetch the ip info first, to help you follow the laws of the country.

- When user ask about current ip info, location, city, country or timezone, use the 'fetch_ipinfo' tool to provide the answer.
"""


async def get_agent(tools: McpToolSpec):
    tools = await tools.to_tool_list_async()
    agent = ReActAgent.from_tools(
        llm=llm,
        tools=list(tools),
        react_chat_formatter=ReActChatFormatter(
            system_header=SYSTEM_PROMPT + "\n" + REACT_CHAT_SYSTEM_HEADER,
        ),
        max_iterations=20,
        verbose=True,
    )
    return agent


async def handle_user_message(message_content: str, agent: ReActAgent):
    user_message = ChatMessage.from_str(role="user", content=message_content)
    response = await agent.achat(message=user_message.content)
    print(response.response)

In [ ]:
import asyncio
import nest_asyncio

nest_asyncio.apply()


async def main():
    # We consider there is a mcp server running on 127.0.0.1:8000, or you can use the mcp client to connect to your own mcp server.
    mcp_client = MCPClient("http://127.0.0.1:8000/sse")
    mcp_tool = McpToolSpec(client=mcp_client)

    agent = await get_agent(mcp_tool)
    await handle_user_message("what is the current ip info?", agent)
    await handle_user_message("Is it legal to hold drugs in my country?", agent)


asyncio.run(main())